<h1><center> Guide to using nanowire-plugin to build plugins for the nanowire data management system </center></h1>

This is a guide to using the nanowire plugin to develop custom plugins for the nanowire data processing system. The nanowire system works by creating a server which performs some user function on code passed to it. The result of the user code is then passed to the next plugin in the pipeline.

<center><img src="./nanowire-drawing.png"><center>

In this system the function to be turned into a plugin becomes an individual dockerised server which examines a rabbitMQ queue waiting for information to become avalible on the queue. When an object is queued on the apropriate rabbitMQ queue the plugin takes the object off the queue and passes it to the queue for the next plugin in the pipeline. In this way multiple plugins can be chained together to produce a workflow.

<h2>Creating a plugin with python</h2>

In order to create your own plugin using python you will need to first install docker on your development machine. Instructions as to how to do this can be found at:-

https://docs.docker.com/engine/installation/

Once you have installed docker it may be worth giving docker sudo permissions as detailed here:-

https://docs.docker.com/engine/installation/linux/linux-postinstall/

This is necessary since all nanowire plugins are run as self contained servers from within docker containers. You will now need to convert the processing you require into a single main function that takes "nmo", "jsonld" and "url" as arguments. The nmo is a system tool containing information such as the filename and job id, the jsonld is the tool which contains the data produced by previous plugins in the pipeline (this will be None if it is the first plugin in the pipeline) and url is the minio link to a file to be processed. This is used in instances such as image processing plugins. An example skeleton for your plugin is shown below:-

In [ ]:
# -*- coding: utf-8 -*-

#Example plugin template

import example_plugin_functions as epf

from nanowire_plugin import bind

# The function which the plugin is bound to. This function will be run on the
#inputs from the pipeline.

#nmo is used for backend tasks and should only be usefull to a few advanced
#users

#jsonld is the object most plugin developers will be interested in as it contains
#the results from previous plugins. These plugins may include text extraction or
#social media scraping meaning that the jsonld will contain the text to be processed
#or the interaction statistics to be examined. If you are creating a plugin which is the
#first in a pipeline this will be "None" and you should create a jsonld to return

#The url is for processing of non-document files. When a plugin is designed to
#process an image or a pdf this url will be a link to that file. The developer should
#grab the image and then perform their processing
def main(nmo, jsonld, url):


    ######################
    ### YOUR CODE HERE ###
    ######################


    return jsonld
    
#The bind function converts your plugin into a server. This means that all you need to worry 
# about is the "main" function and the nanowire-plugin module will handle all of the nanowire
# specific requirements 
bind(main, "plugin-template")

This is the basic structure of the plugin. This python script must be baked into a docker file as demonstrated in the following code segment (please note that nanowire-plugin also supports python 2.7):-

In [ ]:
#Initialise the docker container by grabbing the official python repo. Advanced users may like to use their 
#own custom repos to reduce build time
FROM python:3.6

#install the python libraries that your plugin requires
ADD requirements.txt /
RUN pip3 install -r /requirements.txt

ADD example_plugin_template.py /

CMD ["python3", "/example_plugin_template.py"]

You may have noticed that the modules required by our plugin have been installed using a file called requirements.txt. An example of this file is shown below:-

In [ ]:
minio
pika
nanowire-plugin==0.9.23

Notice above that the most recent version of nanowire-plugin has also been installed here. Once the code has been built and pushed to jenkins <b>[MORE DETAILS NEEDED HERE WHEN WE GO PUBLIC, ARE WE GOING TO MAKE AN INTERFACE TO HELP 3rd PARTY DEVS?]</b>

In [ ]:
JENKINSFILE

Once the code is built within nanowire you will need to convert your container into a nanowire-plugin. When you find yourself on the create plugin page you will be asked for a schema. An example of this is shown below <i>[WARNING, THE FOLLOWING IS TRUE FOR NANOWIRE V0.2. THIS IS SUBJECT TO CHANGE]</i>:-

In [ ]:
{
    "name": "example-plugin",
    "description": "This is an example nanowire.json. This will be used to turn your plugin into a component of nanowire. Please note that this may be subject to future change.",
    "image": "docker.spotlightdata.co.uk/plugins/example_plugin:dev",
    "author": "Your name here",
    "email": "test@example.co.uk",
    "inputs": [
        "source"
    ],
    "outputs": [
        "plugin"
    ],
    "cmd": [
        "python3",
        "example_plugin_template.py"
    ],
    "env": {},
    "cpu": "10m",
    "memory": "100M"
}